In [4]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

In [5]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
df=pd.read_csv(URL)
df.columns

Index(['Date', 'Year', 'Month', 'Recession', 'Consumer_Confidence',
       'Seasonality_Weight', 'Price', 'Advertising_Expenditure', 'Competition',
       'GDP', 'Growth_Rate', 'unemployment_rate', 'Automobile_Sales',
       'Vehicle_Type', 'City'],
      dtype='object')

In [6]:
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True

app.layout = html.Div(id='main', children=[
    html.H1('Automobile Sales Statistics Dashboard', className='H1'),
    html.Div(id='dropDownDiv', children=[
        dcc.Dropdown(id='dropdown-statistics',
                     options=[
                         {'label': 'Yearly Statistics',
                             'value': 'Yearly Statistics'},
                         {'label': 'Recession Period Statistics',
                             'value': 'Recession Period Statistics'}
                     ],
                     placeholder='Select a report type', 
                     value='Select Statistics'
                    ),
        dcc.Dropdown(id='select-year',
                     options=[
                         {'label':i,'value':i} for i in range (1980,2024,1)
                     ],
                     placeholder='Select-year',
                     value='Select-year',
                    )
    ]),
    html.Div(id='output-container', className='chart-grid',style={'display': 'flex'})
]
)



@app.callback(
    Output(component_id='select-year',component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value')
)

def update_input_container(recessionType):
    match recessionType:
        case 'Yearly Statistics': return 0
        case 'Recession Period Statistics': return 1

@app.callback(Output(component_id='output-container',component_property='children'),
              [Input(component_id='dropdown-statistics',component_property='value'),
               Input(component_id='select-year',component_property='value')])

def update_output_container(recessionType,year):
    data=df
    match recessionType:
        case 'Yearly Statistics': 
            data=df[df['Recession']==0]

        case 'Recession Period Statistics': 
            data = df[df['Recession'] == 1]

    #year_data=data[data['Year']==year]
    return html.Div(plot1(data)+plot2(data)+plot3(data)+plot4(data), id='wrapper')
   



    
def plot1(data):
    annualMean = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
    R_chart1 = dcc.Graph(
        figure=px.line(
            annualMean, x='Year', y='Automobile_Sales',
            title='Automobile Sales During Recession Period')
    )
    return [html.Div(R_chart1,className='grids')]
   
def plot2(data):
    avgSalesCount = data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
    R_chart2 = dcc.Graph(
        figure=px.bar(avgSalesCount,
                      x='Vehicle_Type',
                      y='Automobile_Sales',
                      title='Average Sales by Vehicle Type')
    )
    return [html.Div(R_chart2, className='grids')]

def plot3(data):
    expShare = data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
    expShare['Advertising_Expenditure'] = expShare['Advertising_Expenditure']/expShare['Advertising_Expenditure'].sum()
    R_chart3 = dcc.Graph(
        figure=px.pie(
            values=expShare['Advertising_Expenditure'], names=expShare['Vehicle_Type'],
            title='Total Expenditure Share by Vehicle Type During Recessions'
        )
    )
    return [html.Div(R_chart3, className='grids')]

def plot4(data):
    unempData = data.groupby(['Vehicle_Type','unemployment_rate'])['Automobile_Sales'].mean().reset_index()
    R_chart4 = dcc.Graph(figure=px.bar(
        unempData,
        x=unempData['unemployment_rate'],
        y=unempData['Automobile_Sales'],
        color=unempData['Vehicle_Type'],
        labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
        title='Effect of Unemployment Rate on Vehicle Type and Sales')
    )
    return [html.Div(R_chart4, className='grids')]


if __name__ == '__main__':
    app.run_server()